# Fine-tuning a Hugging Face base model using Unsloth and TRL

This is a worked example of how to efficiently fine-tune a base language model from [Hugging Face Hub](https://huggingface.co/models) using the [Unsloth](https://docs.unsloth.ai) and [TRL](https://huggingface.co/docs/trl/en/index) libraries on an instruction-based task.

Unsloth integrates with TRL in order to reduce the time and GPU memory required to fine-tune LLMs, when compared to using TRL exclusively.

To run fine-tuning, you must have first [generated the dataset](https://rasa.com/rasa-pro/docs/operating/fine-tuning-recipe) files `train.jsonl` and `val.jsonl`, which must be in the [TRL instruction format](https://huggingface.co/docs/trl/en/sft_trainer#dataset-format-support).

## 1. Configure fine-tuning environment

This notebook has been tested on on a [GCP Vertex AI Workbench instance](https://cloud.google.com/vertex-ai/docs/workbench/instances/introduction) with machine type `a2-highgpu-1g` in the `asia-southeast1-b` zone, which had the following hardware:
- Single NVIDIA A100 GPU (40GB VRAM)
- 12 core CPU with 85B RAM
- 256GB disk

It has also been tested on on a [AWS Sagemaker Notebook instance](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html) with machine type `ml.g4dn.4xlarge` in the `eu-central-1` region, which had the following hardware:
- Single NVIDIA T4 GPU (16GB VRAM)
- 16 core CPU with 64GB RAM
- 256GB disk

In both setups, the notebook was executed in a Linux environment with the following software already installed:
- Python 3.10
- CUDA Toolkit 12.1
- PyTorch 2.2

In order to run fine-tuning yourself, upload this notebook along with your dataset files to your own environment.

It is highly recommended that you restart your notebook kernel and re-run all notebook cells every time you wish to perform fine-tuning.

> Despite the fact that this notebook will work with a relatively underpowered GPU, such as the NVIDIA T4, running LLM fine-tuning and inference will be very slow.
>
> It is highly recommended that you use an NVIDIA A100 or other similar GPU types.
>
> The code presented here has been configured for use with an NVIDIA A100, please take note of later comments on the changes you should make to the code when using a different type of GPU.


## 2. Install Python requirements

The following `pip` commands will install Unsloth and other required Python packages when [Conda](https://anaconda.org/anaconda/conda) is used to manage your Python environment, which is the case for many Jupyter notebook runtimes.

If Conda is not used in your environment, please follow [these alternative instructions](https://github.com/unslothai/unsloth?tab=readme-ov-file#pip-installation) for installing Unsloth.

In [ ]:
%%sh
# install unsloth and other dependencies
pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
pip install --no-deps "xformers<=0.0.26" "trl<0.9.0" peft accelerate bitsandbytes huggingface_hub[cli]
# remove tpu-only package that is installed by default on gcp runtimes, even when only using gpu
pip uninstall torch-xla -y

## 3. Download base model

You can download the model you want to fine-tune from Hugging Face Hub using the [official CLI](https://huggingface.co/docs/huggingface_hub/en/guides/cli) with an [API access token](https://huggingface.co/docs/transformers.js/en/guides/private#step-1-generating-a-user-access-token) as per the code below. Make sure you first update the `HUGGINGFACE_TOKEN` and `BASE_MODEL` environment variables with your own values.

When testing this notebook, the [Llama 3.1 8B Instruct](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct) model was used. Note that `meta-llama/Meta-Llama-3.1-8B-Instruct` is a [gated model](https://huggingface.co/docs/hub/en/models-gated) that you must first request access to before using. 

You can use any other PyTorch model available on [Hugging Face Hub](https://huggingface.co/models). It is recommended that you use a model that has been pre-trained on instructional tasks, such as the [CodeLlama 13B Instruct](https://huggingface.co/codellama/CodeLlama-13b-Instruct-hf) model.

Pre-trained models with more parameters will generally perform better at tasks than models with fewer parameters. However, the size of model you can use is limited by how much memory your GPU has.

Alternatively, if you already have a PyTorch model directory to hand, you can upload it to your notebook environment manually.

In [ ]:
%%sh
# TODO: update with your values
export HUGGINGFACE_TOKEN="CHANGEME"
export BASE_MODEL="meta-llama/Meta-Llama-3.1-8B-Instruct"

# download model
huggingface-cli download "${BASE_MODEL}" \
    --token "${HUGGINGFACE_TOKEN}" \
    --local-dir "./base_model" \
    --exclude "*.bin*"

## 4. Load and quantize base model

The [quantization of model parameters](https://huggingface.co/docs/optimum/en/concept_guides/quantization) can significantly reduce the GPU memory required to run model fine-tuning and inference, at the cost of model accuracy.

Here, the base model is loaded from disk and quantized into an 8-bit representation on the fly using the [BitsAndBytes](https://huggingface.co/docs/transformers/main/en/quantization/bitsandbytes) library.

If you are using a GPU with relatively little memory, such as the NVIDIA T4, or if you are using a base model larger than `meta-llama/Meta-Llama-3.1-8B-Instruct`, you may be required to use 4-bit quantization (e.g. `load_in_4bit = True`) in order to avoid 
out of memory (OOM) errors.

In [ ]:
from transformers import BitsAndBytesConfig
from unsloth import FastLanguageModel

# configure quantization method for base model
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

# load quantized model and tokenizer from disk
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="./base_model",
    max_seq_length=2048,
    quantization_config=quantization_config,
)

## 5. Load training and validation datasets

The following code loads the training and validation datasets from the `train.jsonl` and `val.json` files, respectively

As the files use the TRL instruction format, the TRL trainer used later will be able to [automatically parse](https://huggingface.co/docs/trl/en/sft_trainer#dataset-format-support) the datasets and [generate the prompts from a template](https://huggingface.co/docs/transformers/en/chat_templating) configured in the tokenizer.

Prompt templates vary between models and TRL will infer the correct template from your base model. If this is not available in your base model or if you wish to change it, you can set your own [template string](https://huggingface.co/docs/transformers/en/chat_templating#advanced-adding-and-editing-chat-templates).

You can also define your own [prompt formatting function](https://huggingface.co/docs/trl/en/sft_trainer#format-your-input-prompts) in order to have full control of how the prompts are constructed.

In [ ]:
import datasets
from trl.extras.dataset_formatting import get_formatting_func_from_dataset

# load datasets from disk
train_dataset = datasets.load_dataset(
    "json", data_files={"train": "train.jsonl"}, split="train"
)
eval_dataset = datasets.load_dataset(
    "json", data_files={"eval": "val.jsonl"}, split="eval"
)

# test prompt templating on example from dataset
print(get_formatting_func_from_dataset(train_dataset, tokenizer)(eval_dataset[0]))

## 6. Configure trainer

Below, the arguments for the supervised fine-tuning (SFT) trainer are configured. Their values were chosen somewhat arbitrarily and resulted in satisfactory results during testing.

It is recommended that you read the official documentation and experiment with the arguments passed to `SFTConfig` (see [here](https://huggingface.co/docs/trl/main/en/sft_trainer#trl.SFTTrainer)) and `SFTTrainer` (see [here](https://huggingface.co/docs/trl/main/en/sft_trainer#trl.SFTTrainer)).

For example:
- If you get an OOM error when running fine-tuning, you can reduce `per_device_train_batch_size` in order to reduce the memory footprint. However, if your GPU has sufficient memory, you can try increasing it in order to reduce the total number of training steps.
- Consider setting `max_steps`, as you may not need to perform all epochs in order to achieve optimal model accuracy. Conversely, you may see better model accuracy by increasing `num_train_epochs`.
- If fine-tuning is taking too long, you can increase `eval_steps` in order to reduce how often validation is performed. 

In [6]:
import torch
from transformers import TrainingArguments
from trl import SFTTrainer

# configure training args
args = TrainingArguments(
    # training
    per_device_train_batch_size=8,
    warmup_steps=50,
    num_train_epochs=4,
    learning_rate=0.0001,
    lr_scheduler_type="cosine",
    optim="adamw_bnb_8bit",
    weight_decay=0.0,
    logging_steps=1,
    # datatypes
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    # evaluation
    eval_strategy="steps",
    eval_steps=50,
    per_device_eval_batch_size=8,
    output_dir="outputs",
)

# setup trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=args,
)

## 7. Perform supervised fine-tuning

In the code below, fine-tuning is performed using the previously congfigured trainer.

When testing this step on an NVIDIA A100 using the configuration defined above, it took around 12 minutes to perform fine-tuning with a training dataset containing around 500 examples.

After fine-tuning, the base model and fine-tuned adapters are [merged together and saved to disk](https://docs.unsloth.ai/basics/saving-models/saving-to-vllm) in 16-bit for future compatibility with the [vLLM](https://github.com/vllm-project/vllm) model serving library.

If you are using a relatively small GPU, such as the NVIDIA T4, you may have to save the model in 4-bit instead (e.g. `save_method = "merged_4bit_forced"`).

In [ ]:
# run fine-tuning
finetune_metrics = trainer.train()

# save model to disk in 16-bit
model.save_pretrained_merged(
    "./finetune_model", tokenizer, save_method="merged_16bit"
)

## 8. Visualize fine-tuning metrics

Some of the metrics collected during fine-tuning are visualised below in order for you to diagnose any potential issues with the model.

Specifically, the training and validation losses are plotted against the training step number. Please check the plot for the following:
- Ideally, as the fine-tuning steps increase, the training and validation losses should decrease and converge. 
- If both loss curves do not converge, it may be worth performing more fine-tuning steps or epochs. This is known as [underfitting](https://www.ibm.com/topics/underfitting).
- If the validation loss suddenly starts to increase while the training loss continues to decrease or converge, you should decrease your total number of steps or epochs. This is known as [overfitting](https://www.ibm.com/topics/overfitting).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# plot step against train and val losses
fig, ax = plt.subplots()
log_history = pd.DataFrame(trainer.state.log_history)
eval_loss = (
    log_history[["step", "eval_loss"]]
    .dropna()
    .plot(x="step", ax=ax)
)
train_loss = (
    log_history[["step", "loss"]]
    .dropna()
    .plot(x="step", ax=ax)
)
fig.show()

## 9. Run ad hoc inference

You can load your fine-tuned model from disk using Unsloth and use it to run optimized inference on individual inputs of your choosing using the code below.

Note that the inputs passed to model are in the [TRL convertsational format](https://huggingface.co/docs/trl/en/sft_trainer#dataset-format-support) as the Hugging Face [chat template requires them to be](https://huggingface.co/docs/transformers/main/en/chat_templating#how-do-i-use-chat-templates). During training TRL will [automatically convert the instruction format to the conversational format](https://github.com/huggingface/trl/blob/main/trl/extras/dataset_formatting.py). However, you have to do this yourself when applying chat templates manually for inference.

In [ ]:
from transformers import TextStreamer
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained("./finetune_model")
FastLanguageModel.for_inference(model)  # enable inference optimizations
streamer = TextStreamer(tokenizer)  # stream model outputs as they are generated

# the content to include in the input prompt
# by default, a value from the validation dataset as example
content = eval_dataset["prompt"][0]

# apply prompt template and tokenize
input_ids = tokenizer.apply_chat_template(
    [{"role": "user", "content": content}],  # in the TRL conversational format
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# generate model output from user input
_ = model.generate(
    input_ids=input_ids,
    streamer=streamer,  # remove streamer if you want whole output at end
    max_new_tokens=64,  # set the limit on how many tokens are generated
    do_sample=False,    # disable random sampling for deterministic outputs
)

## 10. Export fine-tuned model

Lastly, export your fine-tuned model directory to an appropriate storage location that can be easily accessed later for [deployment](https://rasa.com/rasa-pro/docs/building-assistants/self-hosted-llm).

It is recommended that you use a cloud object store, such as [Amazon S3](https://aws.amazon.com/s3/) or [Google Cloud Storage](https://cloud.google.com/storage).

Uncomment and run the corresponding commands below for your cloud provider, making sure to first update the environment variables with your own values. It is assumed that:
- your bucket already exists
- you have already installed the CLI tool for your cloud provider
- you have already authenticated with your cloud provider and have sufficient permissions to write to your bucket

In [ ]:
%%sh
export LOCAL_MODEL_PATH="./finetune_model"

# if using amazon
# export S3_MODEL_URI="s3://CHANGEME" # update with your value
# aws s3 cp "${LOCAL_MODEL_PATH}" "${S3_MODEL_URI}" --recursive

# if using google
# export GCS_MODEL_URI="gs://CHANGEME" # update with your value
# gsutil cp -r "${LOCAL_MODEL_PATH}" "${GCS_MODEL_URI}"